In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
# Gate Definitions
X = np.array([[0, 1], [1, 0]])
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
I = np.eye(2)
CNOT = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 0, 1],
                 [0, 0, 1, 0]])

In [3]:
# Function to apply Kronecker product on gates to match qubit count
def apply_gate(state, gate, target, n):
    op = 1
    for i in range(n):
        if i == target:
            op = np.kron(op, gate)
        else:
            op = np.kron(op, I)
    return np.dot(op, state)

In [4]:
# Initialize state
def initialize_state(n):
    state = np.zeros(2**n)
    state[0] = 1
    return state

# Run Circuit with Matrix Multiplication
def run_circuit(n):
    state = initialize_state(n)
    state = apply_gate(state, H, 0, n)
    if n > 1:
        state = apply_gate(state, CNOT, 0, n)  # CNOT on first two qubits
    return state

In [5]:
# Runtime Plot
qubits = range(1, 15)
times = []
for n in qubits:
    start = time.time()
    run_circuit(n)
    times.append(time.time() - start)

plt.plot(qubits, times)
plt.xlabel('Number of Qubits')
plt.ylabel('Runtime (s)')
plt.show()

ValueError: shapes (8,8) and (4,) not aligned: 8 (dim 1) != 4 (dim 0)